### python scraping test

### Reference:
#### target
JVN iPedia - 脆弱性対策情報データベース
> https://jvndb.jvn.jp/search/index.php?mode=_vulnerability_search_IA_VulnSearch&lang=ja&keyword=Linux&useSynonym=1&vendor=&product=&datePublicFromYear=2020&datePublicFromMonth=12&datePublicToYear=2025&datePublicToMonth=11&dateLastPublishedFromYear=&dateLastPublishedFromMonth=&dateLastPublishedToYear=&dateLastPublishedToMonth=&v3Severity%5B0%5D=01&v3Severity%5B1%5D=02&cwe=&searchProductId=&pageNo=2

#### scrapy
10分で理解する Scrapy #Python - Qiita
> https://qiita.com/Chanmoro/items/f4df85eb73b18d902739

python超初心者がスクレイピングしてみる #Python - Qiita
> https://qiita.com/ritukiii/items/272d485e8a249d0d1bd7

新人「先輩、Scrapyでまだextractやextract_firstを使ってるんですか？😊」 #Python - Qiita
> https://qiita.com/yoshi_10_11/items/0ef017c99a0c4d145d9a

#### RAG
【RAG入門】ソースコード付き：簡単にRAGを始めたい人へ #AI - Qiita
> https://qiita.com/batakeee/items/88141504e1fe0d5b8edc

### install
Supported Python versions are 3.9 ~ 3.13 

python repos:\
pip install scrapy

arch linux:\
sudo pacman -S scrapy

### import

In [6]:
!python -m scrapy version

Scrapy 2.12.0


### start scrapy

In [4]:
!scrapy startproject scrapy_jvn

New Scrapy project 'scrapy_jvn', using template directory '/usr/lib/python3.13/site-packages/scrapy/templates/project', created in:
    /home/yokogawa/ドキュメント/MySeniorThesis-main/scrapy_jvn

You can start your first spider with:
    cd scrapy_jvn
    scrapy genspider example example.com


\> tree\
Permissions Size User     Date Modified Name\
drwxr-xr-x     - user     9 10月 14:21 ./\
drwxr-xr-x     - user     9 10月 14:21 ├── scrapy_jvn/\
drwxr-xr-x     - user     9 10月 14:13 │   ├── spiders/\
.rw-r--r--     0 user     7 12月  2024 │   ├── __init__.py\
.rw-r--r--   265 user     9 10月 14:21 │   ├── items.py\
.rw-r--r--  3.7k user     9 10月 14:21 │   ├── middlewares.py\
.rw-r--r--   363 user     9 10月 14:21 │   ├── pipelines.py\
.rw-r--r--  3.3k user     9 10月 14:21 │   └── settings.py\
.rw-r--r--   263 user     9 10月 14:21 └── scrapy.cfg

### edit ./scrapy_jvn/items.py

In [ ]:
import scrapy

class Post(scrapy.Item):
    title = scrapy.Field()   # jvn's title
    content = scrapy.Field() # cve's contents
    product = scrapy.Field() # cve product status: product

### add and edit spider

In [19]:
!pwd
!ls -l

/home/yokogawa/ドキュメント/MySeniorThesis-main
合計 1476
-rw-r--r-- 1 yokogawa yokogawa  25234  9月 10 17:06 AutoregressiveDecoding.jpg
-rw-r--r-- 1 yokogawa yokogawa  80385  9月 10 17:06 CacheSteering-Fig1.png
-rw-r--r-- 1 yokogawa yokogawa  51847  9月 10 17:06 CacheSteering-Fig2.png
-rw-r--r-- 1 yokogawa yokogawa  40942  9月 10 17:06 CacheSteering-Fig3.png
-rw-r--r-- 1 yokogawa yokogawa 391675  9月 10 17:06 CacheSteering-Table1.png
-rw-r--r-- 1 yokogawa yokogawa 137652  9月 10 17:06 CacheSteering-Table2.png
-rw-r--r-- 1 yokogawa yokogawa  63750  9月 10 17:06 CacheSteering-Table3.png
-rw-r--r-- 1 yokogawa yokogawa  37023  9月 10 17:06 CacheSteering-Table4.png
-rw-r--r-- 1 yokogawa yokogawa 342125  9月 10 17:06 Flamingo_architecture.png
-rw-r--r-- 1 yokogawa yokogawa   5227  9月 11 17:08 LLM_Articles_trend.md
-rw-r--r-- 1 yokogawa yokogawa  13356 10月  3 18:13 LLM_withRLHF.md
-rw-r--r-- 1 yokogawa yokogawa   6678  9月 25 15:37 Linux_and_LLM.md
-rw-r--r-- 1 yokogawa yokogawa    464  9月 11 17:12 README.md


#### Navigate to your Scrapy project directory and execute the following command:
```
scrapy genspider ${spider_name} ${domain_name}
```

ex)
```
scrapy genspider jvn_spider-test https://jvndb.jvn.jp/ja/contents/2025/JVNDB-2025-005907.html
```

or create the file using the following as a reference:

In [ ]:
import scrapy

class JvnSpiderTestSpider(scrapy.Spider):
    name = "jvn_spider-test"
    allowed_domains = ["jvndb.jvn.jp"]
    start_urls = ["https://jvndb.jvn.jp/ja/contents/2025/JVNDB-2025-005907.html"]

    def parse(self, response):

        """
        レスポンスに対するパース処理
        """
        # response.css で scrapy デフォルトの css セレクタを利用できる
        for post in response.css('.post-listing .post-item'):
            # items に定義した Post のオブジェクトを生成して次の処理へ渡す
            yield Post(
                title=post.css('div.post-header a::text').getall(),
                content=post.css('div.post-header span.date a::text').getall(),
                product=post.css('div.post-header span.date a::text').getall(),
            )

        # 再帰的にページングを辿るための処理
        older_post_link = response.css('.blog-pagination a.next-posts-link::attr(href)').extract_first()
        if older_post_link is None:
            # リンクが取得できなかった場合は最後のページなので処理を終了
            return

        # URLが相対パスだった場合に絶対パスに変換する
        older_post_link = response.urljoin(older_post_link)
        # 次のページをのリクエストを実行する
        yield scrapy.Request(older_post_link, callback=self.parse)


### edit ./scrapy_jvn/settings.py

In [ ]:
# Enable the following. Simply uncomment it:
DOWNLOAD_DELAY = 5

HTTPCACHE_ENABLED = True
HTTPCACHE_EXPIRATION_SECS = 0
HTTPCACHE_DIR = 'httpcache'
HTTPCACHE_IGNORE_HTTP_CODES = []
HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

### run scrapy

In [ ]:
!scrapy crawl scrapy jvn_spider-test -o data.csv

### search html/css

In [ ]:
!scrapy shell ${domain_name}

### scraping analysis results

In [1]: response.css('body').getall()
Out[1]: ['<body>\n<a name="top" id="top"></a>\n<!-- ↓ ヘッダ表示 ↓ -->\n<div id="wrapper">\n\n ... <div id="mainarea">\n        <!-- leftarea begin -->\n        <div id="leftarea">\n            <div class="contents">\n                <div id="news-list">\n<br>\n<table class="vuln_table_clase"> ... </body>']

In [2]: response.css('title::text').getall()
Out[2]: ['JVNDB-2025-005907 - JVN iPedia - 脆弱性対策情報データベース']

In [3]: response.css('body a::attr(href)').getall()
Out[3]:
['/',
 ...
 'https://www.cve.org/CVERecord?id=CVE-2025-32434',
 'https://nvd.nist.gov/vuln/detail/CVE-2025-32434',
...]

Scrapyのセレクターでテキストを取得 #Scrapy - Qiita
> https://qiita.com/itoi10/items/ab855a5e2fdad1535931